# **TITLE**

Hello reader, my name is Ben and this is a tutorial on the basics of data science. In this tutorial, we will be...

In [539]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, glob

https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/#:~:text=2.,and%20merge%20them%20into%20DataFrame.

In [540]:
path = './raw-data/'
data = pd.concat(map(pd.read_csv, glob.glob(path + '*.csv')))

data.reset_index(drop=True, inplace=True)
data.columns = ['tour', 'year', 'season', 'event_name', 'event_id', 'player_name', 'player_id', 'finish', 'round', 'course_name', 'course_num', 'course_par', 'start_hole', 'teetime',
                'round_score', 'sg_putt', 'sg_arg', 'sg_app', 'sg_ott', 'sg_t2g', 'sg_total', 'driving_dist', 'driving_acc', 'gir', 'scrambling', 'prox_rgh', 'prox_fw', 'great_shots', 'poor_shots']

data.head()


,tour,year,season,event_name,event_id,player_name,player_id,finish,round,course_name,...,sg_t2g,sg_total,driving_dist,driving_acc,gir,scrambling,prox_rgh,prox_fw,great_shots,poor_shots
0,pga,2021,2021,The Masters #2,536,"Matsuyama, Hideki",13562,1,1,Augusta National Golf Club,...,3.207,5.523,283.8,0.714,0.722,1.000,44.273,38.172,7.0,3.0
1,pga,2021,2021,The Masters #2,536,"Matsuyama, Hideki",13562,1,2,Augusta National Golf Club,...,3.062,1.330,301.9,0.786,0.778,0.429,38.792,38.045,3.0,3.0
2,pga,2021,2021,The Masters #2,536,"Matsuyama, Hideki",13562,1,3,Augusta National Golf Club,...,5.630,7.426,298.5,0.357,0.667,0.714,50.730,10.420,7.0,3.0
3,pga,2021,2021,The Masters #2,536,"Matsuyama, Hideki",13562,1,4,Augusta National Golf Club,...,1.103,-0.370,304.3,0.714,0.611,0.636,166.794,36.691,4.0,5.0
4,pga,2021,2021,The Masters #2,536,"Zalatoris, Will",19428,2,1,Augusta National Golf Club,...,2.985,4.523,302.0,0.643,0.778,0.571,35.775,38.183,6.0,2.0


In [541]:
players = data[['player_name', 'player_id']].drop_duplicates()
players.columns = ['name', 'id']
players = players.set_index('id')

players.head()

,name
id,
13562,"Matsuyama, Hideki"
19428,"Zalatoris, Will"
14636,"Spieth, Jordan"
19895,"Schauffele, Xander"
7649,"Leishman, Marc"


In [542]:

events = data[['event_id', 'event_name', 'course_num', 'course_name', 'course_par', 'year']].drop_duplicates()
events.columns = ['id', 'event_name', 'course_num', 'course_name', 'course_par', 'year']
events = events.reset_index(drop=True)

for event in data['event_id'].unique():
    for year in data['year'].unique():
        if not events.loc[(events['id'] == event) & (events['year'] == year)].empty:
            data.loc[(data['event_id'] == event) & (data['year'] == year), 'event_id'] = events.loc[(events['id'] == event) & (events['year'] == year)].index[0]

events['id'] = events.index
events = events.set_index('id')

events['event_name'] = events['event_name'].apply(lambda x: "The Masters" if "Masters" in x else x)
events['event_name'] = events['event_name'].apply(lambda x: "U.S. Open" if "U.S." in x else x)

events['major'] = events['event_name'].apply(lambda x: 'M' if "Masters" in x else 'U' if "U.S." in x else 'P' if "PGA" in x else 'O')

events

,event_name,course_num,course_name,course_par,year,major
id,,,,,,
0,The Masters,14,Augusta National Golf Club,72,2021,M
1,U.S. Open,654,The Country Club,70,2022,U
2,The Open Championship,668,St. Andrews Links (Old Course),72,2022,O
3,The Open Championship,724,Royal Liverpool,71,2023,O
4,U.S. Open,43,The Los Angeles Country Club (North Course),70,2023,U
5,The Open Championship,778,Royal St. George's GC,70,2021,O
6,The Masters,14,Augusta National Golf Club,72,2023,M
7,The Masters,14,Augusta National Golf Club,72,2022,M
8,PGA Championship,697,Ocean Course at Kiawah Island,72,2021,P


In [543]:
finishes = data[['event_id', 'player_id', 'finish']].drop_duplicates()
finishes.reset_index(drop=True, inplace=True)
finishes['finish'] = finishes['finish'].apply(lambda x: str(x).replace('T', ''))
finishes['finish'] = finishes['finish'].apply(lambda x: np.nan if not x.isdigit() else int(x))
finishes.head()

,event_id,player_id,finish
0,0,13562,1.0
1,0,19428,2.0
2,0,14636,3.0
3,0,19895,3.0
4,0,7649,5.0


In [544]:
data.drop(['year', 'event_name', 'course_name', 'course_par', 'course_num', 'finish', 'tour', 'season', 'start_hole', 'player_name'], axis=1, inplace=True)
data

      event_id  player_id  round  teetime  round_score  sg_putt  sg_arg  \
0            0      13562      1   9:48am           69    2.315   1.793   
1            0      13562      2   1:00pm           71   -1.732   1.479   
2            0      13562      3   1:30pm           65    1.796   1.318   
3            0      13562      4   2:40pm           73   -1.473   1.014   
4            0      19428      1   8:48am           70    1.538  -0.108   
5            0      19428      2  11:54am           68    3.375   0.022   
6            0      19428      3   2:20pm           71    0.294  -0.437   
7            0      19428      4   2:20pm           70   -0.343   0.793   
8            0      14636      1   2:00pm           71   -1.174   2.859   
9            0      14636      2  10:54am           68    0.372   2.454   
10           0      14636      3   2:00pm           72   -2.487   1.088   
11           0      14636      4   2:10pm           70   -0.214   0.396   
12           0      19895

Discuss null values